In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
a = pd.read_csv("../기상청공모전/data/derivate_data.csv")

In [3]:
a1 = a.iloc[:,1:]

In [4]:
a1 = a1[a1["cog"] != -99.9]
a1 = a1[a1["hdg"] != -99.9]
a1 = a1[a1["hdg"] != -99]

In [5]:
# Stochastic Gradient Boosting
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 데이터셋 생성
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Stochastic Gradient Boosting 모델 학습
params = {'objective': 'binary:logistic', 'eval_metric': 'logloss', 'booster': 'gblinear', 'subsample': 0.8}
num_rounds = 100  # 반복 횟수 설정

model = xgb.train(params, dtrain, num_rounds)

# 테스트 데이터 예측
y_pred = model.predict(dtest)
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred]

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
print("정확도:", accuracy)
print("F1 Score:", f1)

[17:15:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "subsample" } are not used.

정확도: 0.9792875959571561
F1 Score: 0.22529644268774704


In [6]:
from sklearn.metrics import confusion_matrix

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred_binary)

# CSI 계산
tp = cm[1, 1]  # True Positive
tn = cm[0, 0]  # True Negative
fp = cm[0, 1]  # False Positive
fn = cm[1, 0]  # False Negative

csi = tp / (tp + fn + fp)
print("CSI 점수:", csi)

CSI 점수: 0.12694877505567928
